In [38]:
def put(code, op, pointer, relative, offset, value):
    mode = op % (10**(2+offset)) // (10**(1+offset))
    if mode == 0: # parameter mode
        ix = code[pointer+offset]
    elif mode == 2: # relative mode
        ix = code[pointer+offset] + relative
    else: raise ValueError(f"{op}, {pointer}, {offset} not allowed")

    if ix >= len(code): # Make sure we allocate enough memory
        code += [0] * (ix - len(code) + 1)
    code[ix] = value

def get(op, code, pointer, relative, offset):
    # e.g. get(1002, code, 2) -> code[p] (immediate mode)
    # e.g. get(2, code, 2) -> code[code[p]] (parameter mode)
    mode = op % (10**(2+offset)) // (10**(1+offset))
    if mode == 0: # parameter mode
        if pointer+offset >= len(code): return 0 if code[0] >= len(code) else code[0]
        if code[pointer+offset] >= len(code): return 0
        return code[code[pointer+offset]]
    elif mode == 1: # immediate mode
        if pointer+offset >= len(code): return 0
        return code[pointer+offset]
    elif mode == 2: # relative mode
        if pointer+offset >= len(code): return 0 if relative+code[0] >= len(code) else code[relative]
        if relative+code[pointer+offset] >= len(code): return 0
        return code[relative+code[pointer+offset]]
    raise ValueError(f"{op}, {pointer}, {offset} not allowed")

def step(code, p, r, io):
    # INPLACE does a intcode step, and returns new state and new p
    op = code[p]
    if op % 100 == 1: # add
        put(code, op, p, r, 3, get(op, code, p, r, 1) + get(op, code, p, r, 2))
        return code, p+4, r, io
    elif op % 100 == 2: # mul
        put(code, op, p, r, 3, get(op, code, p, r, 1) * get(op, code, p, r, 2))
        return code, p+4, r, io
    elif op % 100 == 3: # read
        put(code, op, p, r, 1, io.read())
        return code, p+2, r, io
    elif op % 100 == 4: # write
        return code, p+2, r, io.write(get(op, code, p, r, 1))
    elif op % 100 == 5: # jmp_f
        if get(op, code, p, r, 1) != 0:
            return code, get(op, code, p, r, 2), r, io
        return code, p+3, r, io
    elif op % 100 == 6: # jmp_t
        if get(op, code, p, r, 1) == 0:
            return code, get(op, code, p, r, 2), r, io
        return code, p+3, r, io
    elif op % 100 == 7: # lt
        put(code, op, p, r, 3, int(get(op, code, p, r, 1) < get(op, code, p, r, 2)))
        return code, p+4, r, io
    elif op % 100 == 8: # eq
        put(code, op, p, r, 3, int(get(op, code, p, r, 1) == get(op, code, p, r, 2)))
        return code, p+4, r, io
    elif op % 100 == 9: # relative
        return code, p+2, r + get(op, code, p, r, 1), io
    elif op % 100 == 99: # exit
        return code, -1, r, io
    raise ValueError(f"Incorrect program. Op is {op}")

def run(code, io):
    pointer = 0
    relative = 0
    while pointer >= 0:
        #print(len(code), pointer, relative, io.output(), "next op:", code[pointer:(pointer+4)])
        code, pointer, relative, io = step(code, pointer, relative, io)
    return code, io


In [42]:
class IO():

    def __init__(self, board=None):
        self.board = board if board is not None else {}
        self.position = (0, 0)
        self.rotation = (0, 1)
        self.paint = True # True for paint, False for turn

    def read(self):
        # white is 1, black is 0. Everything starts as black!
        return 0 if self.position not in self.board else self.board[self.position]
    
    def write(self, value):
        if self.paint:
            self.board[self.position] = value
        else:
            # rotate, then move. 0 is left, 1 is right
            self.rotation = (-1*self.rotation[1], self.rotation[0]) if value == 0 else \
                            (self.rotation[1], -1*self.rotation[0])
            self.position = (self.position[0] + self.rotation[0],
                             self.position[1] + self.rotation[1],)
        self.paint = not self.paint
        return self
    
    def output(self):
        return len(self.board)

In [44]:
import advent

data = advent.get_intcode(11)

_, out = run(data.copy(), IO())
out.output()

2441

In [50]:
_, out = run(data.copy(), IO({(0, 0): 1}))

def draw_board(board):
    output = [[board.get((y, x-6), 0) for y in range(0, 43)] for x in range(6, 0, -1)]
    output = [''.join(map(lambda x: '.' if x == 0 else '#', line)) for line in output]
    return output

res = draw_board(out.board)
for line in res:
    print(line)

.###..####.###..####.###..###..#..#..##....
.#..#....#.#..#.#....#..#.#..#.#.#..#..#...
.#..#...#..#..#.###..#..#.#..#.##...#......
.###...#...###..#....###..###..#.#..#......
.#....#....#.#..#....#....#.#..#.#..#..#...
.#....####.#..#.#....#....#..#.#..#..##....
